#  Data Analysis and Results Collection

This file is for automaticly output all the analysis for all dataset at once 

In [1]:
#!pip install xlwt

In [2]:
#Imports
import math
import librosa
import librosa.display
import IPython.display as ipd

import pyAudioAnalysis
import pydub
import ffmpeg

from pydub import AudioSegment
from pydub.silence import split_on_silence

import numpy as np
import matplotlib.pyplot as plt

import xlwt
from xlwt import Workbook

from numpy import sin, cos, pi, linspace, arange, log10, absolute
from numpy.random import randn
from scipy.signal import lfilter, lfilter_zi, filtfilt, butter, freqz, welch
from scipy.io.wavfile import read , write
from scipy.signal import spectrogram
from matplotlib import pyplot
from matplotlib.pyplot import ylabel, xlabel
from matplotlib.pyplot import plot, legend, show, grid, figure, savefig#, hold

C:\Users\Dasil\anaconda3\envs\PDS\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [3]:
import numpy as np
def real_cepstrum(x, n=None):
# Compute the real cepstrum of a real sequence
#    x : ndarray
#        Real sequence to compute real cepstrum of.
#    n : {None, int}, optional
#        Length of the Fourier transform.
#    Returns
#    -------
#    ceps: ndarray
#        The real cepstrum.

    spectrum = np.abs(np.fft.fft(x, n=n))**2
    ceps = np.fft.ifft(np.log(spectrum))

    return ceps, spectrum

In [4]:
#Zero Crossing Rate

def ZCR(samples, frameSize, overlap):
    wlen = len(samples)
    step = frameSize - overlap
    frameNum = math.ceil(wlen/step)
    zcr = np.zeros((frameNum,1))
    for i in range(frameNum):
        curFrame = samples[np.arange(i*step,min(i*step+frameSize,wlen))]
        #To avoid DC bias, usually we need to perform mean substraction on each frame
        curFrame = curFrame - np.mean(curFrame) #Zero-Justified
        zcr[i] = sum(curFrame[0:-1]*curFrame[1::]<=0)
    return zcr

In [5]:
# Workbook is created
wb = Workbook()

#List of Words
words = ["_chata_0","_chapa_0","_chave_0","_lata_0","_lapa_0","_lava_0","_casa_0","_capa_0","_cave_0","_cata_0","_chuta_0","_chupa_0","_chuva_0","_farta_0","_farpa_0","_farda_0","_ripa_0","_rita_0","_rica_0"]

#List of Dataset versions
d = {"one": 1, "two": 2}
iterable = d.keys()

#List of people
p = ['A','J','N']

# add_sheet is used to create sheet.
sheet1 = wb.add_sheet('Pitch Analysis')
  
sheet1.write(1, 0, 'Andre')
sheet1.write(1, 1, 'PW_spec')
sheet1.write(2, 1, 'Ceps')
sheet1.write(3, 1, 'Avg ZCR')
sheet1.write(4, 0, 'Joaquin')
sheet1.write(4, 1, 'PW_spec')
sheet1.write(5, 1, 'Ceps')
sheet1.write(6, 1, 'Avg ZCR')
sheet1.write(7, 0, 'Nuno')
sheet1.write(7, 1, 'PW_spec')
sheet1.write(8, 1, 'Ceps')
sheet1.write(9, 1, 'Avg ZCR')

h = 2

#Write Colum Names
for x in words:
    for item in iterable:
        val = x+str(d[item])
        sheet1.write(0, h, val)
        h += 1
h=1
w=0
flag = False

for y in words:
    for item in iterable:
        h += 1
        if (w == 9):
            w = 0        
        for x in p:
            #Write file path
            l = str(d[item])
            filename = 'audio/training/'+x+y+l+'.wav'
            
            #Read file path
            fs, audio1 = read(filename)
            f,t,S1 = spectrogram(audio1, fs, window='flattop', nperseg=fs//10, noverlap=fs//20, scaling='spectrum', mode='magnitude')

            #Apply butterworth Filter
            wn = 1500/(fs/2)
            b3,a3 = butter(4, wn)
            audio1_filt = lfilter(b3,a3,audio1)

            #Analyze Audio (With audio Segment)
            song = AudioSegment.from_wav(filename)

            #Divide Audio in two
            db = -30
            segments = split_on_silence(song, min_silence_len = 100, silence_thresh = db)
            while len(segments) != 2:
                db = db-1
                segments = split_on_silence(song, min_silence_len = 100, silence_thresh = db)
                if db < -90:
                    break
                    
            #Find out the number of Segments
            n = len(segments)
            #print("Size of List", n)


            # Process each chunk with your parameters
            for i, segment in enumerate(segments):

                # Export the audio chunk with new bitrate.
                #print("Exporting chunk{0}.wav.".format(i))
                segment.export(
                    "audio/data/chunk{0}.wav".format(i),
                    bitrate = "192k",
                    format = "wav"
                )

            samples = segments[0].get_array_of_samples()
            samples = np.array(samples)
            
            # Calculate Power Spectrum & Cepstrum
            ceps, spec = real_cepstrum(samples, n=None)
            N = samples.shape[0]

            # Power Specturm
            power_spec = np.abs(spec[:N//2])**2
            #plt.plot(power_spec[0:3000])

            # Calculate Zero Cross Rate
            frameSize = 256
            overlap = 0
            zcr = ZCR(samples, frameSize, overlap)
            
            index = np.where(power_spec == np.max(power_spec))
            #print('max value index: ', index[0])

            abs_ceps = np.abs(ceps[:N//2])**2
            #plt.plot(abs_ceps[:10])
            np.max(abs_ceps)
            
            #print("Power Spectrum: ", index[0], "\n")
            #print("Power Cepstrum: ", round(np.max(abs_ceps)))
            #print(round(np.average(zcr)))
            #print(b)
            
            w +=1
            sheet1.write(w, h, int(index[0][0]))
            w +=1
            sheet1.write(w, h, round(np.max(abs_ceps)))
            w +=1 
            sheet1.write(w, h, round(np.average(zcr)))
            flag = True

wb.save('audio/DataResults.xls')

C:\Users\Dasil\AppData\Local\Temp/ipykernel_22240/2613002886.py:53: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs, audio1 = read(filename)
